# 02_Filtering TOP texts
My goal here is to retrieve only the TOP that have to do with what we are looking for.

## Disclaimer:

Eu rodei esse código e o final ficou muito ruim, absolutamente impossível de seguir os discursos dessa maneira. 
Eu preciso arrumar um jeito de limitar os tagesordnungpunkt. Aquilo estava completamente misturado.
Talvez eu tenha que fazer isso na mao !

------

The code is focused on creating a regular expression pattern for selecting specific terms related to the field of forestry and wood industry. The regular expressions in the ```search_terms_holz``` list are designed to match words starting with specific prefixes or containing certain keywords. Here is a breakdown of the individual regular expressions:

r'\bholz\w*': Matches words starting with the prefix "holz" followed by zero or more word characters.
r'\bforst\w*': Matches words starting with the prefix "forst" followed by zero or more word characters.
r'\bwald\w*': Matches words starting with the prefix "wald" followed by zero or more word characters.
r'\bbioökonomie': Matches the exact word "bioökonomie".
r'\bbioenergie': Matches the exact word "bioenergie".
r'\bmöbel\w*': Matches words starting with the prefix "möbel" followed by zero or more word characters.


We also selected some ```forbiden_terms```, words that match the regular expressions but known not beeing actual related with forestry or wood industry, for example names (Holzner) or verbs (Waldspaziergang)

In [1]:
import numpy as np
import pandas as pd
import re

tagesordnungpunkt_df = pd.read_csv("tagesordnungpunkt_df.csv", sep=';', encoding='UTF-8')

In [2]:
tagesordnungpunkt_df.head(1)

,Unnamed: 0,id,dokumentart,typ,vorgangsbezug_anzahl,dokumentnummer,wahlperiode,herausgeber,pdf_hash,datum,titel,tagesordnungpunkt,fundstelle,vorgangsbezug,aktualisiert
0,0,5557,Plenarprotokoll,Dokument,55.0,20/91,20.0,BT,acf17f0dcd2b9245bc3b8e70ca673dd0,2023-03-16,Protokoll der 91. Sitzung des 20. Deutschen Bu...,Beginn: 9.00 Uhr\nPräsidentin Bärbel Bas:\nSeh...,{'pdf_url': 'https://dserver.bundestag.de/btp/...,"[{'id': '285357', 'titel': 'Finanzierung des p...",2023-04-05T15:30:47+02:00


In [13]:
# Define the search terms and forbidden terms
search_terms_holz = [r'\bholz\w*', r'\bforst\w*', r'\bwald\w*', r'\bbioökonomie', r'\bbioenergie', r'\bmöbel\w*']

forbidden_terms = ['Holzner', 'Holzleitner', 'Waldneukirchen', 'Holzmann', 'Waldegg', 'Waldheims', 'Waldorf', 'Waldvier', 'Waldei',
                   'Holzkreuze', 'Holzweg', 'Wald4tler', 'Walddorf', 
                  'Waldkirchen', 'Waldburg', 'Waldorfschule', 'Waldspaziergang', 'Holzfuß', 'Waldzell', 'Holzleit',
                  'Waldbe', 'Waldneukirchen', 'Holzgau', 'waldorfmathematisch', 'Waldwürfe', 'Waldviertelbahn', 'Waldheim', 'Walding', 'Waldenstein', 
                  'Waldviertlerin', 'Holzlärm', 'Holzchild', 'Waldviertelautobahn', 'Holzinger', 'Waldviertler', 'Waldbrand', 'Waldbrände', 'Land- und forstwirtschaftliche Landeslehrpersonen-Dienstrechtsgesetz',
                  'Land- und forstwirtschaftliche Landesvertrags­lehr­personengesetz']

# Create the regular expression pattern with word boundaries
padrao_holz = r"\b(" + r"|".join(search_terms_holz) + r")\b"
regex_pattern = re.compile(padrao_holz)

# Filter the DataFrame tagesordnungpunkt_df
filtered_rows = tagesordnungpunkt_df[tagesordnungpunkt_df['tagesordnungpunkt'].str.contains(regex_pattern)]

# Create the DataFrame top_filtered_df
top_filtered_df = pd.DataFrame(filtered_rows)

# Create the 'terms' column with the matched values
top_filtered_df['terms'] = top_filtered_df['tagesordnungpunkt'].apply(lambda x: regex_pattern.findall(x))

# Remove rows from top_filtered_df when 'terms' column is identical to forbidden terms
top_filtered_df = top_filtered_df[~top_filtered_df['terms'].isin(forbidden_terms)]

#creating new index
top_filtered_df['top_index'] = range(len(top_filtered_df))

#Saving:

dir = "C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\New_German_Sten_Protokollen\\TOP_Filtered\\"

for index, row in top_filtered_df.iterrows():
    text = row['tagesordnungpunkt']
    datum = row['datum']
    dokumentnummer = row['dokumentnummer']
    dokumentnummer = re.sub("/", '_', dokumentnummer)
    top_index = str(row['top_index'])

    file_name = dir + datum + '_' + dokumentnummer + '_' + top_index + '.txt'

    # Open the file in write mode
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(text)

    print(file_name , " saved to the file.")

C:\Users\JVoigt\AppData\Local\Temp\ipykernel_22880\321827932.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_rows = tagesordnungpunkt_df[tagesordnungpunkt_df['tagesordnungpunkt'].str.contains(regex_pattern)]


C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\New_German_Sten_Protokollen\TOP_Filtered\2023-03-01_20_87_0.txt  saved to the file.
C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\New_German_Sten_Protokollen\TOP_Filtered\2023-02-08_20_84_1.txt  saved to the file.
C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\New_German_Sten_Protokollen\TOP_Filtered\2023-01-26_20_82_2.txt  saved to the file.
C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\New_German_Sten_Protokollen\TOP_Filtered\2022-12-01_20_73_3.txt  saved to the file.
C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\New_German_Sten_Protokollen\TOP_Filtered\2022-11-30_20_72_4.txt  saved to the file.
C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\New_German_Sten_Protokollen\TOP_Filtered\2022-11-30_